In [11]:
# 환경변수로 설정된 OpenAI API Key 가져오기
import os
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

LLMs - Basics

In [5]:
from langchain.llms import OpenAI

llm = OpenAI()

#1) __call__: string in → string out
llm("Tell me a joke")

#2) generate: batch calls, richer outputs
result = llm.generate(["Tell me a joke", "Tell me a poem"])
result.generations # Be sure to confirm “the structure of generations!!!!”
result.llm_output # meta data of result from llm


{'token_usage': {'prompt_tokens': 8, 'total_tokens': 56, 'completion_tokens': 48}, 'model_name': 'text-davinci-003'}


Async API - for IO Bound Tasks

In [6]:
import asyncio

async def foo():
    print('Running in foo')
    await asyncio.sleep(1)
    print('Explicit context switch to foo again')

async def bar():
    print('Explicit context switch to bar')
    await asyncio.sleep(1)
    print('Implicit context switch to bar')

async def main():
    tasks = [foo(), bar()]
    await asyncio.gather(*tasks)

async def async_generate(llm):
    resp = await llm.agenerate(["Hello, how are you?"])
    print(resp.generations[0][0].text)
    
async def generate_concurrently():
    llm = OpenAI(temperature=0.9)
    tasks = [async_generate(llm) for _ in range(10)]
    await asyncio.gather(*tasks)


await main()
# >>> await generate_concurrently()


Running in foo
Explicit context switch to bar
Explicit context switch to foo again
Implicit context switch to bar


Custom LLM

In [ ]:
from langchain.llms.base import LLM, CallbackManagerForLLMRun
from typing import Any, List, Mapping, Optional
class CustomLLM(LLM):
    n: int

    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        return prompt[: self.n]
    
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"n": self.n}
    
# >>> c_llm = CustomLLM(n=10)
# >>> c_llm('Tell me a joke’)
# 'Tell me a ‘
# >>> c_llm._identifying_params
# {'n': 10}
# >>> c_llm._llm_type
# 'custom'

# 내가 만들 LLM을 사용하는 방법
# LangChain에 툴이 많으니 쓰자

Caching

In [4]:
# Save your money & time

import langchain
from langchain.llms import OpenAI

# To make the caching really obvious, lets use a slower model.
llm = OpenAI(model_name="text-davinci-002", n=2, best_of=2)

# 1. In Memory Cache Method
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()

# The first time, it is not yet in cache, so it should take longer
# >>> llm.predict("Tell me a joke")
# 0.8s in vscode notebook
# The second time it is, so it goes faster
# >>> llm.predict("Tell me a joke")
# 0.0s

# 2. SQLite Cache Method
from langchain.cache import SQLiteCache
langchain.llm_cache = SQLiteCache(database_path=".langchain.db")

llm.predict("Tell me a joke")
# 0.7s
# llm.predict("Tell me a joke")
# 0.0s


'\n\nWhy did the chicken cross the road?\n\nTo get to the other side!'

Streaming Like ChatGPT

In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm = OpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0)

# 1) output will display like ChatGPT display

resp = llm("Write me a song about sparkling water.")
# 2) generate still working with streaming mode
llm.generate(["Tell me a joke."])
# 3) caching still working with streaming mode but immediate retrieval from a current cache
llm("Write me a song about sparkling water.")

Tracking Token Usages

In [13]:
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

llm = OpenAI()

with get_openai_callback() as cb:
    result = llm("Tell me a joke")
    print(cb)

# Tokens Used: 24 Prompt Tokens: 4 Completion Tokens: 20 Successful 
# Requests: 1 Total Cost (USD): $0.00048

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


### Chat Model
LLM에서 발전된게 Chat Model이니 더 많이 쓸 듯

In [19]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

text1 = chat([HumanMessage(content="Translate this sentence from English to Korean: I hate programming.")])

text2 = chat.predict("Translate this sentence from English to Korean: I love programming.")

print(text1)
print()
print(text2)

content='나는 프로그래밍을 싫어해요.' additional_kwargs={} example=False

나는 프로그래밍을 사랑합니다.


chat.generate

In [20]:
# prepare a list of messages
batch_messages = [
[
    SystemMessage(content="You are a helpful assistant that translates English to French."),
    HumanMessage(content="I love programming.")
],
[
    SystemMessage(content="You are a helpful assistant that translates English to French."),
    HumanMessage(content="I love artificial intelligence.")
],
]
result = chat.generate(batch_messages)

# result will display rich information about response messages
result


LLMResult(generations=[[ChatGeneration(text="J'adore programmer.", generation_info={'finish_reason': 'stop'}, message=AIMessage(content="J'adore programmer.", additional_kwargs={}, example=False))], [ChatGeneration(text="J'adore l'intelligence artificielle.", generation_info={'finish_reason': 'stop'}, message=AIMessage(content="J'adore l'intelligence artificielle.", additional_kwargs={}, example=False))]], llm_output={'token_usage': {'prompt_tokens': 53, 'completion_tokens': 18, 'total_tokens': 71}, 'model_name': 'gpt-3.5-turbo'}, run=[RunInfo(run_id=UUID('0f799e5d-a6c6-4cfa-9f75-a7fa9d09cdd5')), RunInfo(run_id=UUID('eab00949-3d78-4455-b88f-152e027dc20d'))])

caching - same as LLMs

In [24]:
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()
#!rm .langchain.db
# We can do the same thing with a SQLite cache
from langchain.cache import SQLiteCache

langchain.llm_cache = SQLiteCache(database_path=".langchain.db")

chat.predict("Translate this sentence from Korean to English: ChatGPT가 SW일자리를 빼앗아 갈 것인가?.")
# 'Will ChatGPT take away software jobs?

'Will ChatGPT take away software jobs?'

prompts for chat model, streaming in chat model 생략? 슥 지나감

Prompt Template vs Chat Prompt Template

In [26]:
# PT
from langchain import PromptTemplate
# 1) get a template from .from_template()
prompt_template = PromptTemplate.from_template(
    template = "Tell me a {adjective} joke about {content}",
)

# 2) get a prompt(str) from .format()
prompt = prompt_template.format(adjective='funny', 
content='chickens')

# 3) type(prompt) → str
# type(prompt)
# >>> str
print(prompt)


Tell me a funny joke about chickens


In [30]:
# CPT

from langchain.prompts import ChatPromptTemplate
# 1) get a template from .from_message()
template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

# 2) get a messages from .format_message()
messages = template.format_messages(
    name="Bob", user_input="What is your name?"
)
# 3) type(messages) → list
# 4) type(element of messages) → System/Human/AI/Human Massage

for mes in messages:
    print(mes)
# print(messages)

content='You are a helpful AI bot. Your name is Bob.' additional_kwargs={}
content='Hello, how are you doing?' additional_kwargs={} example=False
content="I'm doing well, thanks!" additional_kwargs={} example=False
content='What is your name?' additional_kwargs={} example=False


message types for from_messages()

In [31]:
from langchain.prompts.chat import ChatPromptTemplate, SystemMessage, HumanMessagePromptTemplate
template = ChatPromptTemplate.from_messages([
        SystemMessage(
            content=(
            "You are a helpful assistant that re-writes the user's text to sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ])
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI()

llm(template.format_messages(text='i dont like eating tasty things.'))

AIMessage(content='I absolutely adore eating tasty things!', additional_kwargs={}, example=False)

Custom Template
재미 없음 그냥 이렇게 할 수 있다~
generate 함수는 nanoGPT에서 generate 함수 긁어다 넣음

Few-shot Prompt Templates

In [6]:
from langchain import OpenAI
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

examples = [ {"another example of the same format"},{
"question": "Who was the maternal grandfather of George Washington?",
"answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington 
was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington 
was Joseph Ball.
So the final answer is: Joseph Ball
""" },]


llm = OpenAI(model_name="text-davinci-002", n=2, best_of=2)

example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")
prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)
q_str = prompt.format(input="Who was the father of Mary Ball Washington?")
a_str = llm.predict(q_str)
print(a_str)

SyntaxError: invalid syntax (299899643.py, line 4)

Fixed (고정) 되어있는 few-shot examples for chat models